In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
train=pd.read_csv("train.csv")

In [ ]:
def create_dummies(data,column,values,name_prefix=""):
    dummies=pd.get_dummies(data[column])
    print(dummies.columns)
    for col in dummies:
        if(col in values):
            data["{}_{}".format(name_prefix,col)]=dummies[col]


In [ ]:
train.head()


In [ ]:
train.info()

In [ ]:
train.isna().any()

In [ ]:
train.isna().sum()

In [ ]:
feature_column=["length(cm)","height(cm)","X2","condition","X1",'wait_time']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:

train_cp=train[feature_column]
X_train,X_test,y_train,y_test=train_test_split(train_cp,train["pet_category"],train_size=.8,random_state=0)

In [ ]:
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import accuracy_score

In [ ]:
model=DecisionTreeClassifier(random_state=0,max_depth=5,min_samples_leaf=20)
model.fit(X_train,y_train)

In [ ]:
pred_train=model.predict(X_train)
print("train model accuracy=",accuracy_score(y_train,pred_train)*100)
pred_test=model.predict(X_test)
print("test model accuracy=",accuracy_score(y_test,pred_test)*100)

## Feature Engineering

### Condition

In [ ]:
train.fillna(3.0,inplace=True)
train.isna().any()

In [ ]:
pd.options.display.max_rows=1000
train.groupby(['condition','breed_category','pet_category']).size()

### Length & Height

In [ ]:

plt.boxplot("height(cm)",data=train)
plt.show()

In [ ]:
train["length(cm)"]=train["length(m)"]*100.0
plt.boxplot("length(cm)",data=train)
plt.show()

In [ ]:
train[train['length(cm)']==0].groupby(['length(cm)','pet_category']).size()

In [ ]:
val=train["length(cm)"].mean()
val=round(val,2)
train.loc[train["length(cm)"]==0,["length(cm)"]]=val

### X1

In [ ]:
y=train["X1"].value_counts()

plt.bar(y.index,y.values)
plt.xticks(y.index)
plt.figure(figsize=(10,10))
plt.show()

In [ ]:
train.groupby(["X1","pet_category"]).size()

In [ ]:
create_dummies(train,"X1",,"X1_")
train.head()

In [ ]:
create_dummies(train,'X1',[1,7,13,15,17,16,18],'X1')

In [ ]:
train.head()

In [ ]:
train['issue_date']=pd.to_datetime(train['issue_date'])
train['listing_date']=pd.to_datetime(train['listing_date'])

In [ ]:
train['issue_yr']=train.issue_date.dt.year
train['issue_month']=train.issue_date.dt.month
train['listing_yr']=train.listing_date.dt.year


In [ ]:
train['listing_month']=train.listing_date.dt.month
train['wait_time']=train['listing_date'].sub(train['issue_date'])
train['wait_time']=train['wait_time'].dt.days

In [ ]:
train['wait_time'].loc[train['wait_time']<0]=0

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor(n_estimators=300,random_state=0,min_samples_leaf=20)
model2.fit(X_train,y_train)

In [ ]:
pred_train=model2.predict(X_train)
pred_train.astype(int)
print("train model accuracy=",accuracy_score(y_train,pred_train.astype(int))*100)
pred_test=model2.predict(X_test)
print("test model accuracy=",accuracy_score(y_test,pred_test.astype(int))*100)

In [ ]:
from xgboost import XGBClassifier
model3 = XGBClassifier()
model3.fit(X_train,y_train)

In [ ]:
pred_train=model3.predict(X_train)
pred_train.astype(int)
print("train model accuracy=",accuracy_score(y_train,pred_train.astype(int))*100)
pred_test=model3.predict(X_test)
print("test model accuracy=",accuracy_score(y_test,pred_test.astype(int))*100)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
enc=OrdinalEncoder()
enc.fit(train['color_type'].values.reshape(len(train),1))

In [ ]:
t=train.copy()
t=enc.transform(t['color_type'].values.reshape(len(train),1))

In [ ]:
train["encoded"]=t

In [ ]:
train.head()

In [ ]:
enc.categories_